# 🧠 Chapter 3: NLP Tasks using Hugging Face

This notebook will guide you through performing various Natural Language Processing (NLP) tasks using Hugging Face's powerful `pipeline()` function and Transformers library.

---

## 📚 What You'll Learn

| Section | Topic | Description |
|---------|-------|-------------|
| 1 | **Transformer Architecture** | Understanding what makes Transformers special |
| 2 | **The Pipeline Function** | Simplified API for NLP tasks |
| 3 | **Text Classification** | Sentiment analysis & topic classification |
| 4 | **Named Entity Recognition** | Extracting entities from text |
| 5 | **Text Summarization** | Condensing long text into summaries |
| 6 | **Translation** | Converting text between languages |
| 7 | **Text Generation** | Creating new text from prompts |
| 8 | **Question Answering** | Answering questions from context |
| 9 | **Zero-shot Classification** | Classifying without training |

---

## 🔧 Setup

First, let's install and import the necessary libraries.

In [1]:
# Uncomment to install required packages
# %pip install transformers torch accelerate sentencepiece sacremoses -q

In [2]:
# Core imports
import transformers
from transformers import pipeline
import torch
import warnings
warnings.filterwarnings('ignore')

# Check versions
print("📦 Library Versions")
print("=" * 40)
print(f"🤗 Transformers: {transformers.__version__}")
print(f"🔥 PyTorch: {torch.__version__}")

# Device selection
if torch.cuda.is_available():
    device = 0
    print(f"🚀 Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = -1
    print("💻 Using CPU")

📦 Library Versions
🤗 Transformers: 4.57.3
🔥 PyTorch: 2.9.0+cu126
🚀 Using GPU: Tesla T4


---

## 1️⃣ Understanding the Transformer Architecture

The **Transformer** architecture revolutionized NLP in 2017 with the paper "Attention Is All You Need". Here's what makes it special:

### Key Components

| Component | Purpose |
|-----------|--------|
| **Self-Attention** | Allows each word to "look at" all other words in the sentence |
| **Multi-Head Attention** | Multiple attention patterns learned simultaneously |
| **Positional Encoding** | Adds word position information |
| **Feed-Forward Networks** | Processes attention outputs |

### Why Transformers Beat RNNs

```
Traditional RNN:  Word1 → Word2 → Word3 → Word4  (Sequential, slow)
Transformer:      Word1 ⟷ Word2 ⟷ Word3 ⟷ Word4  (Parallel, all at once!)
```

> 💡 **Key Insight**: Transformers can process all words simultaneously, making them much faster and better at capturing long-range dependencies.

### Transformer Architecture Overview

```
┌─────────────────────────────────────────┐
│              INPUT TEXT                 │
│         "The food was amazing"          │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│        TOKENIZATION + EMBEDDING         │
│    Convert words to numerical vectors   │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│         POSITIONAL ENCODING             │
│      Add position information           │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│      TRANSFORMER ENCODER LAYERS         │
│  ┌───────────────────────────────────┐  │
│  │     Multi-Head Self-Attention     │  │
│  └───────────────┬───────────────────┘  │
│                  ▼                      │
│  ┌───────────────────────────────────┐  │
│  │     Feed-Forward Network          │  │
│  └───────────────────────────────────┘  │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│              OUTPUT                     │
│    Sentiment: POSITIVE (0.98)           │
└─────────────────────────────────────────┘
```

---

## 2️⃣ The Pipeline Function

The `pipeline()` function is Hugging Face's **easiest way** to use pre-trained models. It handles:

- ✅ Tokenization (converting text to numbers)
- ✅ Model inference (running the neural network)
- ✅ Post-processing (converting outputs to readable format)

### Available Pipeline Tasks

| Task | Description | Example Use Case |
|------|-------------|------------------|
| `sentiment-analysis` | Positive/negative classification | Product reviews |
| `text-classification` | Multi-class classification | Spam detection |
| `ner` | Named Entity Recognition | Extract names, locations |
| `summarization` | Condense long text | Article summaries |
| `translation_xx_to_yy` | Language translation | Multi-language apps |
| `text-generation` | Generate new text | Creative writing |
| `question-answering` | Answer questions | FAQ bots |
| `zero-shot-classification` | Classify without training | Flexible categorization |

### Basic Usage Pattern

```python
from transformers import pipeline

# Create a pipeline for any task
task_pipeline = pipeline("task-name")

# Use it!
result = task_pipeline("Your input text here")
```

---

## 3️⃣ Text Classification

Text classification assigns categories or labels to text. The most common example is **sentiment analysis**.

### 3.1 Sentiment Analysis - Restaurant Reviews

Let's analyze customer reviews for a restaurant!

In [3]:
# Create a sentiment analysis pipeline
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=device
)

# Restaurant reviews to analyze
restaurant_reviews = [
    "The pasta was absolutely divine! Best Italian food I've had in years.",
    "Service was extremely slow and the waiter was rude. Very disappointed.",
    "Decent food but nothing special. Prices are a bit too high for what you get.",
    "Amazing ambiance and the chef's special was out of this world!",
    "Food poisoning after eating here. Never going back. Avoid at all costs!"
]

print("🍕 Restaurant Review Sentiment Analysis")
print("=" * 60)

for review in restaurant_reviews:
    result = sentiment_analyzer(review)[0]
    emoji = "😊" if result['label'] == 'POSITIVE' else "😞"
    print(f"\n{emoji} {result['label']} ({result['score']:.1%})")
    print(f"   \"{review[:60]}...\"" if len(review) > 60 else f"   \"{review}\"")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


🍕 Restaurant Review Sentiment Analysis

😊 POSITIVE (100.0%)
   "The pasta was absolutely divine! Best Italian food I've had ..."

😞 NEGATIVE (100.0%)
   "Service was extremely slow and the waiter was rude. Very dis..."

😞 NEGATIVE (99.9%)
   "Decent food but nothing special. Prices are a bit too high f..."

😊 POSITIVE (100.0%)
   "Amazing ambiance and the chef's special was out of this worl..."

😞 NEGATIVE (99.8%)
   "Food poisoning after eating here. Never going back. Avoid at..."


### 3.2 Batch Processing for Efficiency

When analyzing many texts, batch processing is much faster!

In [4]:
import time

# More reviews for batch processing demo
many_reviews = restaurant_reviews * 10  # 50 reviews

# Single processing
start = time.time()
single_results = [sentiment_analyzer(r) for r in many_reviews]
single_time = time.time() - start

# Batch processing
start = time.time()
batch_results = sentiment_analyzer(many_reviews)
batch_time = time.time() - start

print("⚡ Batch Processing Performance")
print("=" * 40)
print(f"📊 Processing {len(many_reviews)} reviews")
print(f"\n⏱️ One-by-one: {single_time:.2f}s")
print(f"⏱️ Batch:      {batch_time:.2f}s")
print(f"🚀 Speedup:    {single_time/batch_time:.1f}x faster!")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


⚡ Batch Processing Performance
📊 Processing 50 reviews

⏱️ One-by-one: 0.23s
⏱️ Batch:      0.25s
🚀 Speedup:    0.9x faster!


### 3.3 Multi-class Classification - Tech News Topics

Let's classify technology news headlines into different categories.

In [7]:
# Use zero-shot classification for topic classification
# This allows us to define our own categories without training!
topic_classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=device
)

tech_headlines = [
    "Apple announces new AI-powered chip for next-generation MacBooks",
    "Bitcoin surges past $100,000 as institutional investors pile in",
    "Security researchers discover critical vulnerability in popular browser",
    "SpaceX successfully launches 60 more Starlink satellites into orbit",
    "New study shows social media algorithms impact teen mental health"
]

# Define our custom topic categories
topic_labels = ["AI & Hardware", "Cryptocurrency", "Cybersecurity", "Space Technology", "Social Media"]

print("📰 Tech News Topic Classification")
print("=" * 60)
print(f"🏷️ Categories: {', '.join(topic_labels)}")

for headline in tech_headlines:
    result = topic_classifier(headline, topic_labels)
    top_label = result['labels'][0]
    top_score = result['scores'][0]
    print(f"\n📌 {top_label} ({top_score:.1%})")
    print(f"   {headline}")

Device set to use cuda:0


📰 Tech News Topic Classification
🏷️ Categories: AI & Hardware, Cryptocurrency, Cybersecurity, Space Technology, Social Media

📌 AI & Hardware (92.6%)
   Apple announces new AI-powered chip for next-generation MacBooks

📌 Cryptocurrency (98.8%)
   Bitcoin surges past $100,000 as institutional investors pile in

📌 Cybersecurity (96.2%)
   Security researchers discover critical vulnerability in popular browser

📌 Space Technology (91.4%)
   SpaceX successfully launches 60 more Starlink satellites into orbit

📌 Social Media (98.5%)
   New study shows social media algorithms impact teen mental health


---

## 4️⃣ Named Entity Recognition (NER)

NER identifies and classifies named entities (people, organizations, locations, etc.) in text.

### Entity Types

| Tag | Meaning | Example |
|-----|---------|----------|
| PER | Person | "Elon Musk" |
| ORG | Organization | "Google" |
| LOC | Location | "San Francisco" |
| MISC | Miscellaneous | "iPhone" |

### 4.1 Extracting Entities from Travel Blogs

In [8]:
# Create NER pipeline
ner_pipeline = pipeline(
    "ner",
    model="dbmdz/bert-large-cased-finetuned-conll03-english",
    aggregation_strategy="simple",  # Groups tokens into words
    device=device
)

# Travel blog excerpt
travel_blog = """
Last summer, I traveled from New York to Paris on Air France. 
My guide, Marie Dubois, showed me the Eiffel Tower and the Louvre Museum. 
We had the most amazing croissants at a café near Notre-Dame Cathedral.
I also met fellow travelers from Tokyo and Sydney at our hostel.
"""

print("✈️ Named Entity Recognition - Travel Blog")
print("=" * 60)
print(f"\n📝 Text: {travel_blog.strip()}")
print("\n🏷️ Extracted Entities:")
print("-" * 40)

entities = ner_pipeline(travel_blog)

# Group by entity type
entity_groups = {}
for entity in entities:
    label = entity['entity_group']
    if label not in entity_groups:
        entity_groups[label] = []
    entity_groups[label].append(entity['word'])

# Pretty print
icons = {'PER': '👤', 'LOC': '📍', 'ORG': '🏢', 'MISC': '📦'}
for label, words in entity_groups.items():
    icon = icons.get(label, '🏷️')
    print(f"\n{icon} {label}: {', '.join(set(words))}")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


✈️ Named Entity Recognition - Travel Blog

📝 Text: Last summer, I traveled from New York to Paris on Air France. 
My guide, Marie Dubois, showed me the Eiffel Tower and the Louvre Museum. 
We had the most amazing croissants at a café near Notre-Dame Cathedral.
I also met fellow travelers from Tokyo and Sydney at our hostel.

🏷️ Extracted Entities:
----------------------------------------

📍 LOC: Eiffel Tower, Notre - Dame Cathedral, Tokyo, Louvre Museum, Paris, Sydney, New York

🏢 ORG: Air France

👤 PER: Marie Dubois


### 4.2 Business News Entity Extraction

In [9]:
# Business news excerpts
business_news = [
    "Microsoft CEO Satya Nadella announced a partnership with OpenAI in Seattle.",
    "Amazon is opening a new headquarters in Arlington, Virginia.",
    "Warren Buffett's Berkshire Hathaway increased its stake in Apple Inc."
]

print("💼 Business News Entity Extraction")
print("=" * 60)

for news in business_news:
    print(f"\n📰 {news}")
    entities = ner_pipeline(news)
    
    if entities:
        for entity in entities:
            icon = icons.get(entity['entity_group'], '🏷️')
            print(f"   {icon} {entity['word']} ({entity['entity_group']}, {entity['score']:.1%})")
    print("-" * 40)

💼 Business News Entity Extraction

📰 Microsoft CEO Satya Nadella announced a partnership with OpenAI in Seattle.
   🏢 Microsoft (ORG, 99.9%)
   👤 Satya Nadella (PER, 99.8%)
   🏢 OpenAI (ORG, 99.7%)
   📍 Seattle (LOC, 99.9%)
----------------------------------------

📰 Amazon is opening a new headquarters in Arlington, Virginia.
   🏢 Amazon (ORG, 99.9%)
   📍 Arlington (LOC, 99.7%)
   📍 Virginia (LOC, 99.9%)
----------------------------------------

📰 Warren Buffett's Berkshire Hathaway increased its stake in Apple Inc.
   👤 Warren Buffett (PER, 98.3%)
   🏢 Berkshire Hathaway (ORG, 99.9%)
   🏢 Apple Inc (ORG, 99.9%)
----------------------------------------


---

## 5️⃣ Text Summarization

Text summarization condenses long documents into shorter versions while preserving key information.

### Types of Summarization

| Type | Description |
|------|-------------|
| **Extractive** | Selects and concatenates important sentences |
| **Abstractive** | Generates new text that captures the meaning |

### 5.1 Summarizing Scientific Abstracts

In [10]:
# Create summarization pipeline
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=device
)

# Scientific abstract about climate change
scientific_text = """
Climate change represents one of the most significant challenges facing humanity in the 21st century. 
Rising global temperatures, driven primarily by increased greenhouse gas emissions from human activities, 
are causing widespread environmental changes. These include melting polar ice caps, rising sea levels, 
more frequent and intense weather events, and shifts in ecosystem distributions. Scientists project 
that without significant reductions in carbon emissions, global temperatures could rise by 2-4 degrees 
Celsius by the end of the century. This would result in catastrophic consequences for both natural 
systems and human societies, including food and water scarcity, mass migrations, and economic disruption. 
Addressing climate change requires coordinated global action, including transitioning to renewable energy 
sources, improving energy efficiency, protecting and restoring forests, and developing new technologies 
for carbon capture and storage.
"""

print("🔬 Scientific Text Summarization")
print("=" * 60)
print(f"\n📄 Original ({len(scientific_text.split())} words):")
print(scientific_text.strip())

# Generate summary
summary = summarizer(
    scientific_text, 
    max_length=80, 
    min_length=30, 
    do_sample=False
)[0]['summary_text']

print(f"\n📝 Summary ({len(summary.split())} words):")
print(summary)
print(f"\n📊 Compression: {len(summary.split())/len(scientific_text.split())*100:.0f}% of original")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


🔬 Scientific Text Summarization

📄 Original (128 words):
Climate change represents one of the most significant challenges facing humanity in the 21st century. 
Rising global temperatures, driven primarily by increased greenhouse gas emissions from human activities, 
are causing widespread environmental changes. These include melting polar ice caps, rising sea levels, 
more frequent and intense weather events, and shifts in ecosystem distributions. Scientists project 
that without significant reductions in carbon emissions, global temperatures could rise by 2-4 degrees 
Celsius by the end of the century. This would result in catastrophic consequences for both natural 
systems and human societies, including food and water scarcity, mass migrations, and economic disruption. 
Addressing climate change requires coordinated global action, including transitioning to renewable energy 
sources, improving energy efficiency, protecting and restoring forests, and developing new technologies 
for c

### 5.2 Summarizing with Different Lengths

In [11]:
# Podcast transcript excerpt
podcast_text = """
Today we're discussing the future of artificial intelligence with our special guest, 
a leading researcher in machine learning. We covered three main topics: first, how 
large language models are changing the way we interact with computers, making technology 
more accessible to everyone through natural language interfaces. Second, we explored 
the ethical considerations around AI, including bias in training data, privacy concerns, 
and the potential displacement of workers. Our guest emphasized the importance of 
developing AI responsibly and ensuring that benefits are distributed equitably across 
society. Finally, we looked at emerging applications in healthcare, where AI is being 
used to diagnose diseases earlier, personalize treatments, and accelerate drug discovery. 
The key takeaway is that AI is a powerful tool that requires careful stewardship.
"""

print("🎙️ Podcast Summary - Different Lengths")
print("=" * 60)

# Different summary lengths
lengths = [(30, 50, "Short"), (50, 80, "Medium"), (80, 120, "Detailed")]

for min_len, max_len, label in lengths:
    summary = summarizer(
        podcast_text,
        max_length=max_len,
        min_length=min_len,
        do_sample=False
    )[0]['summary_text']
    
    print(f"\n📌 {label} Summary ({len(summary.split())} words):")
    print(f"   {summary}")

🎙️ Podcast Summary - Different Lengths

📌 Short Summary (43 words):
   A leading researcher in machine learning joined us to discuss the future of artificial intelligence. We looked at emerging applications in healthcare, where AI is being used to diagnose diseases earlier and personalize treatments. The key takeaway is that AI is a powerful

📌 Medium Summary (51 words):
   A leading researcher in machine learning joined us to discuss the future of artificial intelligence. We looked at emerging applications in healthcare, where AI is being used to diagnose diseases earlier, personalize treatments, and accelerate drug discovery. The key takeaway is that AI is a powerful tool that requires careful stewardship.

📌 Detailed Summary (73 words):
   A leading researcher in machine learning joins us to discuss the future of artificial intelligence. We look at how large language models are changing the way we interact with computers. We also explore ethical considerations around AI, including b

---

## 6️⃣ Translation

Translation converts text from one language to another using specialized models.

### 6.1 Translating Cooking Recipes

In [12]:
# English to French translator
translator_en_fr = pipeline(
    "translation_en_to_fr",
    model="Helsinki-NLP/opus-mt-en-fr",
    device=device
)

# Recipe instructions
recipe_steps = [
    "Preheat the oven to 180 degrees Celsius.",
    "Mix flour, sugar, and eggs in a large bowl.",
    "Pour the batter into a greased baking pan.",
    "Bake for 25 minutes until golden brown.",
    "Let it cool before serving with fresh cream."
]

print("🍰 Recipe Translation (English → French)")
print("=" * 60)

for step in recipe_steps:
    translated = translator_en_fr(step)[0]['translation_text']
    print(f"\n🇬🇧 {step}")
    print(f"🇫🇷 {translated}")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


🍰 Recipe Translation (English → French)

🇬🇧 Preheat the oven to 180 degrees Celsius.
🇫🇷 Préchauffer le four à 180 degrés Celsius.

🇬🇧 Mix flour, sugar, and eggs in a large bowl.
🇫🇷 Mélanger la farine, le sucre et les œufs dans un grand bol.

🇬🇧 Pour the batter into a greased baking pan.
🇫🇷 Verser la pâte dans une casserole graissée.

🇬🇧 Bake for 25 minutes until golden brown.
🇫🇷 Cuire au four pendant 25 minutes jusqu'à ce qu'il soit doré.

🇬🇧 Let it cool before serving with fresh cream.
🇫🇷 Laisser refroidir avant de servir avec de la crème fraîche.


### 6.2 Multi-Language Greeting Generator

In [13]:
# Create translators for different languages
translators = {
    "French": pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr", device=device),
    "German": pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de", device=device),
    "Spanish": pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es", device=device)
}

greeting = "Welcome to our restaurant! We hope you enjoy your meal today."

print("🌍 Multi-Language Greeting")
print("=" * 60)
print(f"\n🇬🇧 English: {greeting}")

flags = {"French": "🇫🇷", "German": "🇩🇪", "Spanish": "🇪🇸"}

for language, translator in translators.items():
    translated = translator(greeting)[0]['translation_text']
    print(f"\n{flags[language]} {language}: {translated}")

Device set to use cuda:0


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


🌍 Multi-Language Greeting

🇬🇧 English: Welcome to our restaurant! We hope you enjoy your meal today.

🇫🇷 French: Bienvenue dans notre restaurant! Nous espérons que vous apprécierez votre repas aujourd'hui.

🇩🇪 German: Willkommen in unserem Restaurant! Wir hoffen, dass Sie Ihre Mahlzeit heute genießen.

🇪🇸 Spanish: ¡Bienvenidos a nuestro restaurante! Esperamos que disfruten de su comida hoy.


---

## 7️⃣ Text Generation

Text generation creates new text based on a prompt. This is the technology behind ChatGPT and similar tools.

### 7.1 Creative Story Continuation

In [14]:
# Create text generation pipeline
generator = pipeline(
    "text-generation",
    model="gpt2",
    device=device
)

story_prompt = "The old lighthouse keeper looked out at the stormy sea and noticed something strange. A mysterious light appeared on the horizon, growing brighter with each passing moment."

print("📖 Creative Story Continuation")
print("=" * 60)
print(f"\n✏️ Prompt: {story_prompt}")
print("\n📚 Generated Continuation:")

# Generate continuation
generated = generator(
    story_prompt,
    max_length=150,
    num_return_sequences=1,
    temperature=0.8,
    do_sample=True,
    pad_token_id=generator.tokenizer.eos_token_id
)[0]['generated_text']

# Show only the new part
continuation = generated[len(story_prompt):]
print(continuation)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


📖 Creative Story Continuation

✏️ Prompt: The old lighthouse keeper looked out at the stormy sea and noticed something strange. A mysterious light appeared on the horizon, growing brighter with each passing moment.

📚 Generated Continuation:
 It was this light that he expected when it appeared. It was the power of the sea. There was nothing here that he could not see, but then he thought. The lighthouse keeper had seen the light with his own eyes. He could not deny it was there. He was sure that he had seen it before before, but what he had was a dark and dangerous place. He had seen all the strange lights that people had seen here.

As the lighthouse keeper looked at the lighthouse, he could see nothing. He could not help but sigh. He was so worried. He had called for help. He had even called for something. They were safe. He had to go.

He felt the sunlight on his hand. It was warm and gentle, but there was something missing from it. He felt a little bit shaky, but he did it. He felt

### 7.2 Temperature Effects on Generation

The `temperature` parameter controls creativity:
- **Low (0.1-0.5)**: More predictable, focused
- **High (0.8-1.5)**: More creative, diverse

In [15]:
prompt = "The future of technology is"

print("🌡️ Temperature Effects on Text Generation")
print("=" * 60)
print(f"\n✏️ Prompt: \"{prompt}\"")

temperatures = [0.3, 0.7, 1.2]
labels = ["Conservative", "Balanced", "Creative"]

for temp, label in zip(temperatures, labels):
    generated = generator(
        prompt,
        max_length=60,
        temperature=temp,
        do_sample=True,
        pad_token_id=generator.tokenizer.eos_token_id
    )[0]['generated_text']
    
    print(f"\n🌡️ Temperature {temp} ({label}):")
    print(f"   {generated}")

Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🌡️ Temperature Effects on Text Generation

✏️ Prompt: "The future of technology is"


Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🌡️ Temperature 0.3 (Conservative):
   The future of technology is not yet clear, but it is clear that the future of the Internet is not yet finished.

The Internet is a great thing. It is a great way to connect people and create a better world. It is a great way to connect people and create a better world. It is a great way to connect people and create a better world.

The Internet is a great thing. It is a great thing. It is a great thing. It is a great thing.

It is a great thing. It is a great thing. It is a great thing.

It is a great thing. It is a great thing. It is a great thing.

It is a great thing. It is a great thing. It is a great thing.

It is a great thing. It is a great thing. It is a great thing.

It is a great thing. It is a great thing. It is a great thing.

It is a great thing. It is a great thing. It is a great thing.

It is a great thing. It is a great thing. It is a great thing.

It is a great thing. It is a great thing. It is a great thing


Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🌡️ Temperature 0.7 (Balanced):
   The future of technology is increasingly important, with the potential to revolutionise the way we live and work. Technology makes things better, cheaper and more efficient. As we move into the next decade, we can expect to see more and more connected devices, and more and more connected devices.

With the advent of smart phones, we will be able to interact with each other, interact with the world around us and even communicate in ways that we can't normally do, such as by sending and receiving messages. The use of this technology has the potential to revolutionise the way we work and we are just beginning to see it happen.

Smart phones have helped to democratise how we work, and have brought about the change we need. In light of the latest developments, we need to ensure that we are using smart phones in the right ways.

The world is changing and we need to be making those changes fast. We need to be on the forefront of this revolution, and we need 

### 7.3 Product Description Generator

In [16]:
product_prompts = [
    "Introducing our new wireless headphones:",
    "Experience our premium organic coffee:",
    "Discover the revolutionary fitness tracker:"
]

print("🛍️ Product Description Generator")
print("=" * 60)

for prompt in product_prompts:
    generated = generator(
        prompt,
        max_length=80,
        temperature=0.7,
        do_sample=True,
        pad_token_id=generator.tokenizer.eos_token_id
    )[0]['generated_text']
    
    print(f"\n📦 {generated}")
    print("-" * 40)

Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🛍️ Product Description Generator


Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📦 Introducing our new wireless headphones: the HVAC-F3.

The HVAC-F3 is one of our most popular wireless headphones. It offers a low-power bandwidth and robust performance, making it ideal for casual listening and casual driving. Compared with the older HVAC-F3 models, the HVAC-F3 has a more spacious configuration, which makes it sound more spacious. With an integrated Bluetooth 4.0 interface, we can stream music from any Bluetooth headphones, including the HTC OneM9, HTC One, and the S9.

The HVAC-F3's audio quality is excellent, with high definition sound reproduction that's balanced and balanced. The built-in mic and low-level volume controls are also great, saving you from being overwhelmed when playing music.

The HVAC-F3's wireless headphones are rated for 30 dB of power, which is equivalent to a typical headphone of its class.

Connectivity Features

Each HVAC-F3 has a 12-pin power connector that plugs into any USB port available in the device. The USB port is located on the fr

Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📦 Experience our premium organic coffee: we make our own coffee and then buy it from our farmer's market. We sell our organic coffee in our local farmer's market, and we sell it to our customers for free. If you are a coffee aficionado, your choice will be the most important thing you can do. We are a global company with a diverse portfolio of products and services that are all available at our farm in New Zealand.

All our coffees are produced in the USA, and our coffeehouses in Australia have been around for thousands of years. Our coffeehouse is in the heart of New Zealand's capital city and is the world's largest coffee production facility.

Our coffeehouses are the result of a partnership between the British Government's Government Coffee Plant and the New Zealand Government and the New Zealand Government's Government Coffee Plant, with the objective of providing the world's largest coffee plant with the best quality coffee for export.

The New Zealand Government's Government Cof

---

## 8️⃣ Question Answering

Question Answering (QA) extracts answers from a given context. The model finds the most relevant span of text that answers the question.

### 8.1 Travel Destination Q&A

In [17]:
# Create QA pipeline
qa_pipeline = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad",
    device=device
)

# Travel destination context
japan_context = """
Japan is an island country in East Asia, located in the Pacific Ocean. 
Tokyo is the capital and largest city, with a population of over 13 million people. 
The best time to visit Japan is during spring (March to May) for cherry blossoms 
or autumn (September to November) for colorful foliage. Japan is famous for its 
traditional cuisine including sushi, ramen, and tempura. The country has a rich 
cultural heritage with beautiful temples, shrines, and gardens. Mount Fuji, 
standing at 3,776 meters, is the highest mountain and a UNESCO World Heritage site. 
The currency used is the Japanese Yen (JPY).
"""

questions = [
    "What is the capital of Japan?",
    "When is the best time to visit Japan?",
    "What is the currency in Japan?",
    "How tall is Mount Fuji?",
    "What food is Japan famous for?"
]

print("🗾 Travel Q&A: Japan")
print("=" * 60)
print(f"\n📖 Context: {japan_context.strip()[:200]}...")
print("\n❓ Questions & Answers:")
print("-" * 40)

for question in questions:
    result = qa_pipeline(question=question, context=japan_context)
    print(f"\n❓ {question}")
    print(f"✅ {result['answer']} (confidence: {result['score']:.1%})")

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


🗾 Travel Q&A: Japan

📖 Context: Japan is an island country in East Asia, located in the Pacific Ocean. 
Tokyo is the capital and largest city, with a population of over 13 million people. 
The best time to visit Japan is during spri...

❓ Questions & Answers:
----------------------------------------

❓ What is the capital of Japan?
✅ Tokyo (confidence: 99.6%)

❓ When is the best time to visit Japan?
✅ spring (March to May (confidence: 17.5%)

❓ What is the currency in Japan?
✅ Japanese Yen (confidence: 51.2%)

❓ How tall is Mount Fuji?
✅ 3,776 meters (confidence: 96.3%)

❓ What food is Japan famous for?
✅ sushi, ramen, and tempura (confidence: 57.4%)


### 8.2 Building a Simple FAQ Bot

In [19]:
# Company FAQ context
faq_context = """
TechCorp Support FAQ:

Return Policy: Customers can return products within 30 days of purchase for a full refund. 
Items must be in original packaging and unused condition. Electronics must include all accessories.

Shipping: Standard shipping takes 5-7 business days. Express shipping is available for 
an additional $15 and takes 2-3 business days. Free shipping is offered on orders over $50.

Warranty: All electronic products come with a 2-year manufacturer warranty covering 
defects in materials and workmanship. Accidental damage is not covered.

Contact: Customer support is available 24/7 via live chat on our website. 
Phone support is available Monday through Friday, 9 AM to 6 PM EST at 1-800-TECHCORP.
Email support typically responds within 24 hours at support@techcorp.example.
"""

def ask_faq(question):
    """Simple FAQ bot function"""
    result = qa_pipeline(question=question, context=faq_context)
    return result['answer'], result['score']

print("🤖 TechCorp FAQ Bot")
print("=" * 60)

customer_questions = [
    "How long do I have to return a product?",
    "How much is express shipping?",
    "What does the warranty cover?",
    "When is phone support available?"
]

for q in customer_questions:
    answer, confidence = ask_faq(q)
    print(f"\n👤 Customer: {q}")
    print(f"🤖 Bot: {answer}")
    print(f"   (Confidence: {confidence:.1%})")

🤖 TechCorp FAQ Bot

👤 Customer: How long do I have to return a product?
🤖 Bot: 30 days
   (Confidence: 47.3%)

👤 Customer: How much is express shipping?
🤖 Bot: $15
   (Confidence: 75.6%)

👤 Customer: What does the warranty cover?
🤖 Bot: defects in materials and workmanship
   (Confidence: 95.2%)

👤 Customer: When is phone support available?
🤖 Bot: Monday through Friday
   (Confidence: 68.1%)


---

## 9️⃣ Zero-shot Classification

Zero-shot classification can categorize text into classes **without requiring training examples**. This is incredibly powerful for flexible categorization.

### 9.1 Customer Feedback Classification

In [20]:
# Create zero-shot classifier
zero_shot = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=device
)

# Customer feedback samples
customer_feedback = [
    "My package arrived damaged and the customer service was unhelpful.",
    "Absolutely love this product! It exceeded all my expectations.",
    "The app crashes every time I try to check out. Please fix this bug.",
    "Can you add a dark mode feature? The bright screen hurts my eyes.",
    "Just wondering when the new collection will be available?"
]

# Labels without any training!
feedback_labels = ["complaint", "praise", "bug report", "feature request", "question"]

print("📋 Customer Feedback Classification (Zero-shot)")
print("=" * 60)
print(f"\n🏷️ Categories: {', '.join(feedback_labels)}")
print("-" * 40)

for feedback in customer_feedback:
    result = zero_shot(feedback, feedback_labels)
    top_label = result['labels'][0]
    top_score = result['scores'][0]
    
    print(f"\n📝 \"{feedback[:60]}...\"" if len(feedback) > 60 else f"\n📝 \"{feedback}\"")
    print(f"   → {top_label.upper()} ({top_score:.1%})")

Device set to use cuda:0


📋 Customer Feedback Classification (Zero-shot)

🏷️ Categories: complaint, praise, bug report, feature request, question
----------------------------------------

📝 "My package arrived damaged and the customer service was unhe..."
   → COMPLAINT (77.3%)

📝 "Absolutely love this product! It exceeded all my expectation..."
   → PRAISE (90.7%)

📝 "The app crashes every time I try to check out. Please fix th..."
   → COMPLAINT (63.0%)

📝 "Can you add a dark mode feature? The bright screen hurts my ..."
   → FEATURE REQUEST (45.3%)

📝 "Just wondering when the new collection will be available?"
   → QUESTION (74.6%)


### 9.2 Social Media Post Intent Classification

In [21]:
# Social media posts
social_posts = [
    "Just finished an amazing 10K run! 🏃‍♂️ New personal best!",
    "Anyone else frustrated with the new update? It's so slow!",
    "Check out my new tutorial on machine learning basics! Link in bio.",
    "Grateful for my amazing team. Couldn't have done it without you all! 🙏",
    "Flash sale! 50% off all items this weekend only!"
]

# Intent labels
intent_labels = [
    "personal achievement",
    "expressing frustration", 
    "content promotion",
    "gratitude",
    "sales/marketing"
]

print("📱 Social Media Intent Classification")
print("=" * 60)

for post in social_posts:
    result = zero_shot(post, intent_labels)
    
    print(f"\n💬 {post}")
    print("   Top 3 Intents:")
    for label, score in zip(result['labels'][:3], result['scores'][:3]):
        bar = "█" * int(score * 20)
        print(f"   {bar} {label}: {score:.1%}")

📱 Social Media Intent Classification

💬 Just finished an amazing 10K run! 🏃‍♂️ New personal best!
   Top 3 Intents:
   ███████████████████ personal achievement: 95.6%
    gratitude: 2.4%
    content promotion: 1.7%

💬 Anyone else frustrated with the new update? It's so slow!
   Top 3 Intents:
   ███████████████████ expressing frustration: 98.3%
    content promotion: 0.7%
    sales/marketing: 0.6%

💬 Check out my new tutorial on machine learning basics! Link in bio.
   Top 3 Intents:
   ██████ content promotion: 33.2%
   ████ expressing frustration: 23.4%
   ████ personal achievement: 22.6%

💬 Grateful for my amazing team. Couldn't have done it without you all! 🙏
   Top 3 Intents:
   ███████████████████ gratitude: 97.1%
    content promotion: 1.3%
    sales/marketing: 0.7%

💬 Flash sale! 50% off all items this weekend only!
   Top 3 Intents:
   ██████████████ sales/marketing: 74.7%
   ████ content promotion: 20.3%
    personal achievement: 2.1%


### 9.3 Multi-label Classification

Zero-shot can also handle cases where text belongs to **multiple categories**.

In [22]:
# News article that could belong to multiple categories
article = """
Tech giant Apple announced today that it will invest $5 billion in green energy 
initiatives to power its data centers with 100% renewable energy by 2025. 
The company's stock rose 3% on the news, reaching an all-time high.
"""

categories = ["technology", "business", "environment", "politics", "sports"]

print("🏷️ Multi-label Classification")
print("=" * 60)
print(f"\n📰 Article: {article.strip()}")

# Get all scores
result = zero_shot(article, categories, multi_label=True)

print("\n📊 Category Scores:")
for label, score in zip(result['labels'], result['scores']):
    bar = "█" * int(score * 30)
    status = "✅" if score > 0.5 else "  "
    print(f"{status} {label:15} {bar} {score:.1%}")

# Show assigned labels
assigned = [l for l, s in zip(result['labels'], result['scores']) if s > 0.5]
print(f"\n🏷️ Assigned labels: {', '.join(assigned)}")

🏷️ Multi-label Classification

📰 Article: Tech giant Apple announced today that it will invest $5 billion in green energy 
initiatives to power its data centers with 100% renewable energy by 2025. 
The company's stock rose 3% on the news, reaching an all-time high.

📊 Category Scores:
✅ technology      █████████████████████████████ 98.9%
✅ business        ███████████████████████████ 91.9%
✅ environment     ██████████████████████████ 87.2%
   politics         0.2%
   sports           0.1%

🏷️ Assigned labels: technology, business, environment


---

## 🎯 Conclusion & Key Takeaways

### Summary of NLP Tasks

| Task | Use Case | Key Model |
|------|----------|----------|
| Text Classification | Sentiment, spam, topic | DistilBERT, BERT |
| NER | Extract entities | BERT-large-NER |
| Summarization | Condense text | BART, T5 |
| Translation | Language conversion | Helsinki-NLP models |
| Text Generation | Creative writing | GPT-2, GPT-Neo |
| Question Answering | FAQ bots | DistilBERT-SQuAD |
| Zero-shot | Flexible classification | BART-MNLI |

### Best Practices

1. **Use `pipeline()` for Quick Prototypes**: It's the fastest way to get started
2. **Batch Process When Possible**: Much faster than one-by-one
3. **Choose Task-Specific Models**: Fine-tuned models outperform general ones
4. **Consider Model Size**: Smaller models (DistilBERT) are faster, larger ones more accurate
5. **Use GPU Acceleration**: Significant speedups for larger batches

### Next Steps

- 📚 Explore the [Hugging Face Model Hub](https://huggingface.co/models)
- 🔧 Learn to fine-tune models on custom datasets
- 🖼️ Explore computer vision tasks with Hugging Face

---

**Happy NLP-ing! 🚀**